In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [ ]:
data = pd.read_csv('combined_data.csv')

In [ ]:
# Load the network traffic data
data = pd.read_csv('combined_data.csv')

# Preprocess the data
le = LabelEncoder()
label=le.fit_transform(data['Protocol'])
data['Protocol']=label
data = data.drop(data.columns[0], axis=1)
data.describe()
data=data.dropna()
corr_matrix = data.corr()
corr_matrix
X = data[['Time','Protocol','Length']]
y = data['Target']
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)
importances = rf.feature_importances_
indices = sorted(range(len(importances)), key=lambda i: importances[i], reverse=True)

X=data[['Time','Length']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled
X
y
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,train_size=0.8)
X_train = np.asarray(X_train)
X_test= np.asarray(X_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

C:\Users\Asus\AppData\Local\Temp\ipykernel_8996\2490196129.py:11: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = data.corr()


In [ ]:
# Reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build the model
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)
threshold = 0.5

# Convert predicted values to binary classification
y_pred_binary = (y_pred >= threshold).astype(int)

# Convert actual values to binary classification
y_test_binary = (y_test >= threshold).astype(int)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_binary, y_pred_binary)

print("Accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/3
7931/7931 [==============================] - 60s 7ms/step - loss: 0.0081
Epoch 2/3
7931/7931 [==============================] - 57s 7ms/step - loss: 0.0022
Epoch 3/3
7931/7931 [==============================] - 56s 7ms/step - loss: 0.0020
MSE: 0.0014621821868847874
Accuracy: 99.85%
